In [1]:
import numpy as np
import os
import cv2
import keras
import matplotlib.pyplot as plt
from imutils import paths
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt

Using TensorFlow backend.


###  Loading and Resizing of the data

In [2]:
classes = ['c0', 'c1','c2','c3','c4','c5','c6','c7','c8','c9']
Dict = {'c0' : 0, 'c1' :1, 'c2':2, 'c3':3, 'c4':4,'c5':5,'c6':6,'c7':7,'c8':8,'c9':9}
images = []
Img_labels = []
train_path='imgs/train'

In [3]:
for label in classes:
      path = os.path.join(train_path , label)
      for img in os.listdir(path):
         img = cv2.imread(os.path.join(path,img))
         new_img = cv2.resize(img, (64, 64))
         images.append(new_img)
         Img_labels.append(Dict[label])
         # print(images,Img_labels)
     #print(label)

In [4]:
img=np.array(images)
labels=np.array(Img_labels)
print(img.shape)

(22424, 64, 64, 3)


### Feature extraction using color histogram

In [5]:
# applying color histogram using function for all the images
def color_hist(images):
    histograms = []
    for img in images:
        histograms.append(cv2.calcHist([img], [0, 1, 2],None, [8,8,8], [0, 256, 0, 256, 0, 256]).flatten())
    
    result = np.array(histograms)
        
    return result

In [6]:
x_train,X_test,y_train,Y_test = train_test_split(img,labels, test_size=0.2, random_state=0,stratify=labels)
X_train,X_val,Y_train,Y_val = train_test_split(x_train,y_train, test_size=0.1, random_state=0,stratify=y_train)

In [7]:
train_img = color_hist(X_train)
val_img=color_hist(X_val)
test_img= color_hist(X_test)

In [8]:
val_img.shape, train_img.shape

((1794, 512), (16145, 512))

### Normalizing

In [9]:
from sklearn.preprocessing import MinMaxScaler
def normalize_fn(train,val,test):
    min_max_scaler = preprocessing.MinMaxScaler()
    normalize_train_img = min_max_scaler.fit_transform(train)
    normalize_val_img = min_max_scaler.transform(val)
    normalize_test_img = min_max_scaler.transform(test)
    
    return normalize_train_img, normalize_val_img, normalize_test_img

In [10]:
normalize_train_img, normalize_val_img, normalize_test_img = normalize_fn(train_img,val_img,test_img)

### PCA

In [11]:
from sklearn.decomposition import PCA
pca= PCA(n_components=100)
pca_train_img = pca.fit_transform(normalize_train_img)
pca_val_img = pca.transform(normalize_val_img)
pca_test_img = pca.transform(normalize_test_img)

In [31]:
#using pca
from sklearn.svm import SVC
pca_model = SVC(kernel='rbf')
pca_model.fit(pca_train_img, Y_train)
pca_model_accuracy=pca_model.score(pca_val_img,Y_val)
print("Accuracy on pca",pca_model_accuracy)

Accuracy on pca 0.927536231884058


### LDA

In [12]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA()
lda_train_img = lda.fit_transform(normalize_train_img,Y_train)
lda_val_img = lda.transform(normalize_val_img)
lda_test_img = lda.transform(normalize_test_img)

In [30]:
#using lda
from sklearn.svm import SVC
lda_model = SVC(kernel='rbf')
lda_model.fit(lda_train_img, Y_train)
lda_model_accuracy=lda_model.score(lda_val_img,Y_val)
print("Accuracy on lda",lda_model_accuracy)

Accuracy on lda 0.5668896321070234


### Lda over Pca

In [32]:
lda = LDA()
lda_over_pca_train_img = lda.fit_transform(pca_train_img,Y_train)
lda_over_pca_val_img = lda.transform(pca_val_img)
lda_over_pca_test_img = lda.transform(pca_test_img)

In [27]:
model_lda_over_pca = SVC()
model_lda_over_pca.fit(lda_over_pca_train_img, Y_train)
accuracy_lda_over_pca = model_lda_over_pca.score(lda_over_pca_val_img,Y_val)
print("Lda over Pca model accuracy",accuracy_lda_over_pca)

Lda over Pca model accuracy 0.5306577480490524


In [33]:
pca_model_accuracy=pca_model.score(pca_test_img,Y_test)
print("Accuracy on pca",pca_model_accuracy)

accuracy_lda_over_pca = model_lda_over_pca.score(lda_over_pca_test_img,Y_test)
print("Lda over Pca model accuracy",accuracy_lda_over_pca)

lda_model_accuracy=lda_model.score(lda_test_img,Y_test)
print("Accuracy on lda",lda_model_accuracy)

Accuracy on pca 0.9302118171683389
Lda over Pca model accuracy 0.5351170568561873
Accuracy on lda 0.570345596432553
